# Non-Linear Regression Tree
In the previous notebook the notion of cyclic function for fitting the data for prediction had come up and this is an apparent method used. This notebook will focus on trying to do non-linear regression.

In [1]:
import pandas as pd #DataFrames
df = pd.read_csv('../data/processed/Usable_Historical_Data.csv')
df.dtypes

Unnamed: 0            int64
Product_Code         object
Warehouse            object
Product_Category     object
Date                 object
Order_Demand          int64
Year                float64
Month               float64
Day                 float64
dtype: object

In [2]:
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

Unnamed: 0                   int64
Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand                 int64
Year                       float64
Month                      float64
Day                        float64
dtype: object

In [3]:
df.head()

,Unnamed: 0,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Day
0,0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0,27.0
1,1,Product_0979,Whse_J,Category_028,2012-01-19,500,2012.0,1.0,19.0
2,2,Product_0979,Whse_J,Category_028,2012-02-03,500,2012.0,2.0,3.0
3,3,Product_0979,Whse_J,Category_028,2012-02-09,500,2012.0,2.0,9.0
4,4,Product_0979,Whse_J,Category_028,2012-03-02,500,2012.0,3.0,2.0


In [4]:
df = df.drop('Unnamed: 0', 1)

In [5]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Day
0,Product_0993,Whse_J,Category_028,2012-07-27,100,2012.0,7.0,27.0
1,Product_0979,Whse_J,Category_028,2012-01-19,500,2012.0,1.0,19.0
2,Product_0979,Whse_J,Category_028,2012-02-03,500,2012.0,2.0,3.0
3,Product_0979,Whse_J,Category_028,2012-02-09,500,2012.0,2.0,9.0
4,Product_0979,Whse_J,Category_028,2012-03-02,500,2012.0,3.0,2.0


I'm going off the initial data exploration in assuming that the data is stationary as cyclic behavior as well as static behavior was observed in the resultant graphs displayed below with no ever increasing upward/downward trend:

![title](../reports/figures/Category_Monthly_Bar_Plot.png)

In [6]:
df.set_index('Date')

,Product_Code,Warehouse,Product_Category,Order_Demand,Year,Month,Day
Date,,,,,,,
2012-07-27,Product_0993,Whse_J,Category_028,100,2012.0,7.0,27.0
2012-01-19,Product_0979,Whse_J,Category_028,500,2012.0,1.0,19.0
2012-02-03,Product_0979,Whse_J,Category_028,500,2012.0,2.0,3.0
2012-02-09,Product_0979,Whse_J,Category_028,500,2012.0,2.0,9.0
2012-03-02,Product_0979,Whse_J,Category_028,500,2012.0,3.0,2.0
...,...,...,...,...,...,...,...
2016-04-27,Product_1791,Whse_J,Category_006,1000,2016.0,4.0,27.0
2016-04-27,Product_1974,Whse_J,Category_006,1,2016.0,4.0,27.0
2016-04-28,Product_1787,Whse_J,Category_006,2500,2016.0,4.0,28.0


In [7]:
features = df.join(pd.get_dummies(df['Product_Code']))
target = df['Order_Demand']
features = features.join(pd.get_dummies(df['Year']))
features = features.join(pd.get_dummies(df['Month']))
features = features.join(pd.get_dummies(df['Product_Category']))
features = features.drop(['Product_Code','Product_Category','Order_Demand','Warehouse','Year','Month','Day','Date'],1)
features.head()

,Product_0001,Product_0002,Product_0003,Product_0004,Product_0005,Product_0006,Product_0007,Product_0008,Product_0009,Product_0010,...,Category_024,Category_025,Category_026,Category_027,Category_028,Category_029,Category_030,Category_031,Category_032,Category_033
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Using all categorically focused data we can try and predict the target value of Order_Demand. I dropped warehouses entirely as I believe the warehouse has no sway in the overall demand of the product.

In [8]:
from sklearn.model_selection import train_test_split #Train Test Split
x_train, x_test, y_train, y_test = train_test_split(features.values, target.values, test_size=0.30, random_state=0)

In [9]:
print(x_train.shape); print(x_test.shape)

(734002, 2212)
(314573, 2212)


Well, gonna hope a decision tree can pick up on trends based on features

In [12]:
from sklearn.tree import DecisionTreeRegressor #Decision Tree Regressor for modeling
model_dt = DecisionTreeRegressor(random_state=0)
model_dt.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

Going to use the R2 score as a simple gauge of the accuracy of the model

In [13]:
from sklearn.metrics import r2_score #R2_Score function
predicted_x_train = model_dt.predict(x_train)
r2_score(y_train, predicted_x_train)

0.18828207324478607

This is a real rough R2 Score. Im thinking limiting the max_depth might improve the overall prediction capabilities.

In [14]:
model_dt = DecisionTreeRegressor(max_depth=8,random_state=0)
model_dt.fit(x_train, y_train)
predicted_x_train = model_dt.predict(x_train)
r2_score(y_train, predicted_x_train)

0.12363232781411826

Now that we have a base depth, I'll try a depth of 4 and a depth of 12 to see if we can get better results.

In [15]:
model_dt = DecisionTreeRegressor(max_depth=4,random_state=0)
model_dt.fit(x_train, y_train)
predicted_x_train = model_dt.predict(x_train)
r2_score(y_train, predicted_x_train)

0.07715524244275429

In [16]:
model_dt = DecisionTreeRegressor(max_depth=12,random_state=0)
model_dt.fit(x_train, y_train)
predicted_x_train = model_dt.predict(x_train)
r2_score(y_train, predicted_x_train)

0.1416108507376479

Looks like as the max_depth increases the R2 is mildly higher but not significantly enough. I'm going to serialize this last model for completion sake if the model was wished to be explored more thoroughly in the future.

In [17]:
import pickle #To serialize the model
pickle.dump(model_dt, open('../models/Feature_Decision_Tree.sav', 'wb'))

# Final Thoughts
---

The model generated with Decision Trees isn't accurate and doesn't provide a useful prediction. A different modeling method would ideally be better but the tricky part to overcome is that there are thousands of products that all need to be predicted.  
  
  Even though the categorical data exploration performed showed that there was spikes and drops in Order_Demand data and that not all of the data was periodic, some being static, it might be that the categorical data might be the direction to trim the dataset into usable pieces for prediction.  
  
  An import consideration is if every product needs to have it's demand predicted as some don't exist in the dataset for long periods of time and may be tainting the overall analysis. A future model might take all products with no Order_Demand in 2016 and set their predicted demand to 0 and then only predict for demand of products that were demanded in 2016.